In [ ]:
import numpy as np

def preprocess_embed(x, y):
    """
    Preprocess x and y
    :param x: Feature List of sentences
    :param y: Label List of sentences
    :return: Tuple of (Preprocessed x, Preprocessed y, x tokenizer, y tokenizer)
    """
    preprocess_x, x_tk = tokenize(x)
    preprocess_y, y_tk = tokenize(y)

    preprocess_x = pad(preprocess_x)
    preprocess_y = pad(preprocess_y)
    
    preprocess_target_y = np.roll(preprocess_y,1)
    #preprocess_target_y[-1]=0
    
    # Keras's sparse_categorical_crossentropy function requires the labels to be in 3 dimensions
    #preprocess_y = preprocess_y.reshape(*preprocess_y.shape, 1)

    return preprocess_x, preprocess_y, preprocess_target_y, x_tk, y_tk

preproc_english_sentences_embed, preproc_french_sentences_embed, preproc_frech_sentences_target_embed, english_tokenizer_embed, french_tokenizer_embed =\
    preprocess_embed(english_sentences, french_sentences)

print('Data Preprocessed for embedding')

In [ ]:
#Encoder Decoder Implemetation following the guide by Francois Chollett https://blog.keras.io/a-ten-minute-introduction-to-sequence-to-sequence-learning-in-keras.html

from keras.models import Model
from keras.layers import Input, LSTM, Dense, Embedding, Flatten
import numpy as np

#Encoder 
num_encoder_tokens = 233
latent_dim = 32
num_decoder_tokens = 256
batch_size=128
epochs = 5

tmp_x = pad(preproc_english_sentences, preproc_french_sentences.shape[1])
tmp_x = tmp_x.reshape((-1, preproc_french_sentences.shape[-2]))

print(preproc_english_sentences_embed.shape)

encoder_input_data = preproc_english_sentences_embed
decoder_input_data = preproc_french_sentences_embed
decoder_target_data = preproc_frech_sentences_target_embed

print(preproc_english_sentences_embed.shape)
print(preproc_french_sentences_embed.shape)
# Define an input sequence and process it.
encoder_inputs = Input(shape=(None,))
x = Embedding(num_encoder_tokens, latent_dim)(encoder_inputs)
x, state_h, state_c = LSTM(latent_dim,
                           return_state=True)(x)
encoder_states = [state_h, state_c]

# Set up the decoder, using `encoder_states` as initial state.
decoder_inputs = Input(shape=(None,))
x = Embedding(num_decoder_tokens, latent_dim)(decoder_inputs)
x = LSTM(latent_dim, return_sequences=False)(x, initial_state=encoder_states)
decoder_outputs = Dense(num_decoder_tokens, activation='softmax')(x)

# Define the model that will turn
# `encoder_input_data` & `decoder_input_data` into `decoder_target_data`
model = Model([encoder_inputs, decoder_inputs], decoder_outputs)

# Compile & run training
model.compile(optimizer='rmsprop', loss='categorical_crossentropy')
# Note that `decoder_target_data` needs to be one-hot encoded,
# rather than sequences of integers like `decoder_input_data`!
model.fit([encoder_input_data, decoder_input_data], decoder_target_data,
          batch_size=batch_size,
          epochs=epochs,
          validation_split=0.2)